In [1]:
# Celda 1: Importaciones Necesarias
import requests
import json
import os

print("Librerías importadas correctamente.")

Librerías importadas correctamente.


In [7]:
# Celda 2: Configuración Global
BASE_URL = "http://localhost:8000"  # Asegúrate de que el puerto coincida
#BASE_URL = "https://pdf2image.qaiaanalytics.com"  # Asegúrate de que el puerto coincida
OCR_ENDPOINT_URL = f"{BASE_URL}/convert-pdf"

# Lista de archivos PDF de prueba (colócalos en el mismo directorio que este notebook)
# Puedes añadir más nombres de archivo aquí.
PDF_FILENAMES_TO_TEST = [
    "HAU_800099860_FEHP755052.pdf",
    # "otro_documento.pdf", # Descomenta y añade más PDFs para probar
    # "documento_multiples_paginas.pdf"
]

print(f"Endpoint a probar: {OCR_ENDPOINT_URL}")
print(f"Archivos PDF que se intentarán probar: {PDF_FILENAMES_TO_TEST}")

Endpoint a probar: http://localhost:8000/convert-pdf
Archivos PDF que se intentarán probar: ['HAU_800099860_FEHP755052.pdf']


In [4]:
# Celda 3: Función para Probar el Endpoint /convert-pdf con un Archivo Específico
def probar_ocr_de_pdf(pdf_filename: str):
    """
    Sube un archivo PDF al endpoint /convert-pdf y muestra la respuesta.
    """
    print(f"\n--- Probando OCR para el archivo: {pdf_filename} ---")
    
    pdf_file_path = os.path.join(os.getcwd(), pdf_filename)
    
    if not os.path.exists(pdf_file_path):
        print(f"  ❌ ERROR: El archivo PDF '{pdf_file_path}' no fue encontrado. Omitiendo esta prueba.")
        return None # Retorna None si el archivo no existe

    print(f"  Ruta completa: {pdf_file_path}")
    
    try:
        # Abrir el archivo PDF en modo binario para la subida
        with open(pdf_file_path, 'rb') as f:
            # 'files' es un diccionario donde la clave 'file' debe coincidir
            # con el nombre del parámetro File(...) en tu endpoint FastAPI.
            # La tupla contiene: (nombre_de_archivo_original, objeto_archivo, tipo_mime)
            files_payload = {'file': (pdf_filename, f, 'application/pdf')}
            
            print(f"  Enviando solicitud POST a {OCR_ENDPOINT_URL}...")
            response_ocr = requests.post(OCR_ENDPOINT_URL, files=files_payload, timeout=60) # Timeout de 60 segundos
            
            # Verificar si la solicitud fue exitosa (código 2xx)
            response_ocr.raise_for_status() 
            
            # Obtener la respuesta JSON
            ocr_data = response_ocr.json()
            
            print(f"\n  [Respuesta de /convert-pdf para '{pdf_filename}' - Éxito (Estado {response_ocr.status_code})]")
            print(  "  -------------------------------------------------------------------")

            # 'pages' es una LISTA de diccionarios, cada uno representando una página.
            lista_de_paginas_info = ocr_data.get('pages', [])

            #  Construir el texto completo concatenando el texto de cada página
            textos_de_paginas = []
            if isinstance(lista_de_paginas_info, list): # Asegurarse de que es una lista
                for pagina_info in lista_de_paginas_info:
                    if isinstance(pagina_info, dict) and 'text' in pagina_info: # Asegurarse de que es un dict con 'text'
                        textos_de_paginas.append(pagina_info.get('text', ''))
            # Unir texto de páginas con dos saltos de línea para simular separación de páginas
            texto_completo_extraido = "\n\n".join(textos_de_paginas)
            
            #  El 'detalle_por_pagina' es directamente la lista_de_paginas_info
            detalle_por_pagina = lista_de_paginas_info
            # --- FIN DE AJUSTES ---
            
            print(f"  Texto Completo OCR (primeros 300 caracteres): ' {texto_completo_extraido[:300].replace('', '  ')}'...")

            
            if detalle_por_pagina:
                print(f"\n  Detalle por página (mostrando hasta 2 páginas):")
                for i, page_data in enumerate(detalle_por_pagina):
                    if i < 2: # Limitar la salida
                        print(f"    Página {page_data.get('page')}: '{page_data.get('text', '')[:100].replace('', ' ')}...'")
                    elif i == 2:
                        print(f"    ... y {len(detalle_por_pagina) - 2} página(s) más.")
                        break
            else:
                print("  No se encontró detalle por página o está vacío.")
            
            if texto_completo_extraido.strip():
                print("\n  ✅ Prueba de OCR Parece Exitosa: Se extrajo texto.")
            else:
                print("\n  ⚠️ Prueba de OCR: El endpoint respondió correctamente, pero no se extrajo texto significativo.")
            
            return ocr_data # Devuelve los datos para posible uso posterior

    except requests.exceptions.Timeout:
        print(f"  [ERROR DE TIMEOUT] La solicitud a {OCR_ENDPOINT_URL} para '{pdf_filename}' tardó demasiado.")
    except requests.exceptions.ConnectionError:
        print(f"  [ERROR DE CONEXIÓN] No se pudo conectar a la API en {BASE_URL}.")
        print("  Asegúrate de que el servidor FastAPI (uvicorn) esté ejecutándose.")
    except requests.exceptions.HTTPError as http_err:
        print(f"  [ERROR HTTP: {http_err.response.status_code}] Para '{pdf_filename}'. Respuesta del servidor: {http_err.response.text}")
    except json.JSONDecodeError:
        print(f"  [ERROR DE JSON] La respuesta del servidor para '{pdf_filename}' no es un JSON válido.")
        print(f"  Respuesta recibida: {response_ocr.text if 'response_ocr' in locals() and response_ocr else 'No response object'}")
    except Exception as e:
        print(f"  [ERROR INESPERADO] Ocurrió un error procesando '{pdf_filename}': {e}")
        import traceback
        traceback.print_exc()
    return None # Retorna None si hay error

In [8]:
# Celda 4: Ejecutar las Pruebas de OCR para cada Archivo PDF Definido

print("\n" + "="*10 + " INICIANDO PRUEBAS DEL ENDPOINT /convert-pdf " + "="*10)

resultados_ocr = {} # Diccionario para guardar los resultados si se quieren usar después

if not PDF_FILENAMES_TO_TEST:
    print("No se han definido archivos PDF en 'PDF_FILENAMES_TO_TEST'. Añade nombres de archivo para probar.")
else:
    for pdf_file in PDF_FILENAMES_TO_TEST:
        resultado = probar_ocr_de_pdf(pdf_file)
        if resultado:
            resultados_ocr[pdf_file] = resultado # Guardar el resultado si fue exitoso

print("\n" + "="*10 + " FIN DE LAS PRUEBAS DEL ENDPOINT /convert-pdf " + "="*10)

# Opcional: Mostrar un resumen de los textos extraídos (si se guardaron)
if resultados_ocr:
    print("\nResumen de Textos Extraídos (si alguno fue exitoso):")
    for filename, data in resultados_ocr.items():
        print(f"  Archivo: {filename} - Texto (primeros 50 chars): '{data.get('texto_completo_ocr', '')[:50].replace('',' ')}...'")


========== INICIANDO PRUEBAS DEL ENDPOINT /convert-pdf ==========

--- Probando OCR para el archivo: HAU_800099860_FEHP755052.pdf ---
  Ruta completa: c:\repositorios\pdf2Image\HAU_800099860_FEHP755052.pdf
  Enviando solicitud POST a http://localhost:8000/convert-pdf...

  [Respuesta de /convert-pdf para 'HAU_800099860_FEHP755052.pdf' - Éxito (Estado 200)]
  -------------------------------------------------------------------
  Texto Completo OCR (primeros 300 caracteres): '   @     E  .  S  .  E     H  O  S  P  I  T  A  L     S  A  N     R  A  F  A  E  L     D  E     P  A  C  H  O  
  
  P  p     8  0  0  0  9  9  8  6  0     F  e  c  h  a     A  c  t  u  a  l     :     m  a  r  t  e  s  ,     2  6     a  g  o  s  t  o     2  0  2  5  
  
  P  a  g  i  n  a     1  /  0  
  y     A  T  E  N  C  I  O  N     D  E     U  R  G  E  N  C  I  A  S     R  E  F     1  3  0  5  -  2  6  
  H  O  S  P  I  T  A  L  
  
  S  A  N     R  A  F  A  E  L     D  E     P  A  C  H  O  
  
  N  °     H  i 

In [1]:
for filename, data in resultados_ocr.items():
        print(f"  Archivo: {filename} - Texto (primeros 50 chars): '{data.get('texto_completo_ocr', '')[:100].replace('',' ')}...'")

NameError: name 'resultados_ocr' is not defined